In [4]:
import cv2
import urllib.request
import requests
import numpy as np
import socket
from datetime import datetime
import pandas as pd

http_request_ON = False
socket_ON = True

#http_request_ON = True
#socket_ON = False

# Load the face detection model
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Define the ESP32 Wrover's IP address and port number
url = 'http://10.0.0.15:81'

arr_socket = np.array([])
arr_http = np.array([])

# Open the video stream from the ESP32 Wrover
stream = urllib.request.urlopen(url)

cv2.namedWindow('Video Stream', cv2.WINDOW_NORMAL)

# Loop through the frames in the video stream
while True:
    # Read the next frame from the video stream
    buffer = bytes()
    while True:
        buffer += stream.read(2500) 
        #buffer += stream.read(10000)
        a = buffer.find(b'\xff\xd8')
        b = buffer.find(b'\xff\xd9')
        #print(a, b)
        if a != -1 and b != -1:
            break
        elif a != -1 and b == -1:
            buffer = buffer[a:]
            break
    
    if a != -1 and b != -1 and b > a:
        #print(a,b)
        frame = cv2.imdecode(np.frombuffer(buffer[a:b+2], dtype=np.uint8), cv2.IMREAD_COLOR)

        if frame is not None:
            cv2.imshow('Video Stream', frame)

            # Apply the face detection model to the frame
            faces = face_cascade.detectMultiScale(frame, scaleFactor=1.1, minNeighbors=5)
            
            # Draw bounding boxes around the detected faces
            for (x, y, w, h) in faces:
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
                
                #SET TIMESTAMP, FACE DETECTED
                dt = datetime.now()
                ts_detect = datetime.timestamp(dt)
                #print("FACE DETECTED ON:", ts_detect)
                
                # Define the ESP32's IP address and port number
                ip = "10.0.0.15"
                port = 81
    
                #print("I see you!")

                # Define the integer to send
                message = 3
                
                if http_request_ON == True :
                    # Send number 32 to ESP32 over WiFi
                    print("sending data over http_request!")
                    #requests.get('http://10.0.0.15/?data=3')
                    try:
                        response = requests.get('http://10.0.0.15/?data=3', timeout=1)
                    except requests.Timeout:
                        print("Request timed out")
        
                    #SET TIMESTAMP HTTP_REQUEST
                    dt = datetime.now()
                    ts_HTTP_REQUEST_data = datetime.timestamp(dt)
                    print("DATA SEND VIA HTTP_REQUEST ON:", ts_HTTP_REQUEST_data)
                    
                    diff_data_HTTPREQUEST =  ts_HTTP_REQUEST_data -ts_detect
                    #if statement groter dan 1 sec niet gebruiken in array.
                    if(diff_data_HTTPREQUEST  < 2):
                        arr_http = np.insert(arr_http, 0, [diff_data_HTTPREQUEST * 1000])
                        #print(arr_http)
                        arr_avg = np.array([])
                        arr_avg = np.insert(arr_avg, 0, np.average(arr_http))
                        print("the average of the array is: " , np.average(arr_http) , "(filtered outliers)")
                    print("TIME DIFFERENCE HTTP REQUEST:" , diff_data_HTTPREQUEST * 1000 , "ms") 
                    df = pd.DataFrame({'Time socket' : arr_socket, 'Time detect': ts_detect, 'Difference (ms)' : diff_data_socket})
                    df.to_excel('output.xlsx', sheet_name='Sheet1')
                    
                #define staat bovenaan! 
                if socket_ON == True :
                    # Create a socket object
                    #print("sending data over socket!")
                    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

                    # Connect to the ESP32
                    #print("connecting so ESP32")
                    client_socket.connect((ip, port))

                    # Send the integer as a string
                    #print("sending message as a string")
                    client_socket.send(str(message).encode())
                
                    #SET TIMESTAMP SOCKET
                    dt = datetime.now()
                    ts_socket_data = datetime.timestamp(dt)
                    #print("DATA SEND VIA SOCKET ON:", ts_socket_data)

                    # Close the socket connection
                    client_socket.close()
                    
                    diff_data_socket = ts_socket_data - ts_detect
                    if(diff_data_socket  < 1):
                        arr_socket = np.insert(arr_socket, 0, [diff_data_socket * 1000])
                        #print(arr_socket)
                        arr_avg = np.array([])
                        arr_avg = np.insert(arr_avg, 0, np.average(arr_socket))
                        print("the average of the array is: " , np.average(arr_socket) , "(filtered outliers)")
                    print("TIME DIFFERENCE SOCKET: " , diff_data_socket* 1000 , "ms")
                    df = pd.DataFrame({'Time socket' : arr_socket, 'Time detect': ts_detect,'Difference (ms)' : diff_data_socket})
                    df.to_excel('output.xlsx', sheet_name='Sheet1')
                           
                                                  
            # Display the output
            cv2.imshow('frame', frame)
                        

    # Exit if the 'q' key is pressed
    if cv2.waitKey(1) == ord('q'):
        df.to_excel('output.xlsx', sheet_name='Sheet1')
        break
        
    # Remove the processed data from the buffer
    buffer = buffer[b+2:]


# Release the video stream and close the window
cv2.destroyAllWindows()


the average of the array is:  43.00713539123535 (filtered outliers)
TIME DIFFERENCE SOCKET:  43.00713539123535 ms
the average of the array is:  26.504039764404297 (filtered outliers)
TIME DIFFERENCE SOCKET:  10.000944137573242 ms
the average of the array is:  21.330753962198894 (filtered outliers)
TIME DIFFERENCE SOCKET:  10.984182357788086 ms
the average of the array is:  18.20659637451172 (filtered outliers)
TIME DIFFERENCE SOCKET:  8.834123611450195 ms
